# 2. Word embedding model training

Following approach in [tutorial](https://rare-technologies.com/word2vec-tutorial/).

In [1]:
from gensim.parsing.preprocessing import remove_stopword_tokens

In [29]:
STOPWORDS = """
a about above across after afterwards again against all almost alone along already also although always am among amongst amoungst amount an and another any anyhow anyone anything anyway anywhere are around as at back be
became because become becomes becoming been before beforehand behind being below beside besides between beyond bill both bottom but by call can
cannot cant co computer con could couldnt cry de describe
detail did didn do does doesn doing don done down due during
each eg eight either eleven else elsewhere empty enough etc even ever every everyone everything everywhere except few fifteen
fify fill find fire first five for former formerly forty found four from front full further get give go
had has hasnt have he hence her here hereafter hereby herein hereupon hers herself him himself his how however hundred i ie
if in inc indeed interest into is it its itself keep last latter latterly least less ltd
just
kg km
made make many may me meanwhile might mill mine more moreover most mostly move much must my myself name namely
neither never nevertheless next nine no nobody none noone nor not nothing now nowhere of off
often on once one only onto or other others otherwise our ours ourselves out over own part per
perhaps please put rather re
quite
rather really regarding
same say see seem seemed seeming seems serious several she should show side since sincere six sixty so some somehow someone something sometime sometimes somewhere still such system take ten
than that the their them themselves then thence there thereafter thereby therefore therein thereupon these they thick thin third this those though three through throughout thru thus to together too top toward towards twelve twenty two un under
until up unless upon us used using
various very very via
was we well were what whatever when whence whenever where whereafter whereas whereby wherein whereupon wherever whether which while whither who whoever whole whom whose why will with within without would yet you
your yours yourself yourselves

lrb rrb lcb rcb lsb rsb
"""
STOPWORDS = frozenset(w for w in STOPWORDS.split() if w)
# TODO: remove removal of good stopwords (e.g. he) 

In [30]:
from gensim.test.utils import datapath
from gensim import utils

class TrainingCorpus:
    def __iter__(self):
        corpus_path = datapath('/Users/andrew.wang/Documents/academy/project/kainosRecruitmentApi-TeamA/ai/data/corpus/dataset_wikibios_merged.txt')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            tokens = utils.simple_preprocess(line)
            tokens = remove_stopword_tokens(tokens, stopwords=STOPWORDS)
            yield tokens

Test tokenizer

In [31]:
corpus = TrainingCorpus()
for i,sentence in enumerate(corpus):
    print(sentence)
    if i==5: break

['leonard', 'shenoff', 'randle', 'born', 'february', 'major', 'league', 'baseball', 'player', 'round', 'pick', 'washington', 'senators', 'secondary', 'phase', 'june', 'major', 'league', 'baseball', 'draft', 'tenth', 'overall']
['philippe', 'adnot', 'born', 'august', 'rhèges', 'member', 'senate', 'france', 'elected', 'represents', 'aube', 'department', 'farmer', 'profession', 'serves', 'independent', 'serves', 'head', 'general', 'council', 'aube', 'elected', 'represent', 'canton', 'méry', 'sur', 'seine', 'elected', 'senate', 'round', 'avoiding', 'need', 'run', 'vote', 'having', 'contributed', 'creation', 'university', 'technology', 'troyes', 'vice', 'president', 'university', 'board', 'currently', 'president', 'member', 'senate', 'committee', 'laws', 'relating', 'freedoms', 'universities', 'serves', 'delegate', 'administrative', 'meeting', 'senators', 'list', 'group', 'decorated', 'chevalier', 'ordre', 'national', 'mérite', 'agricole']
['miroslav', 'popov', 'born', 'june', 'dvůr', 'král

In [ ]:
remove numbers, punctuation, stop words and lemmatize?

In [ ]:
import gensim.models

sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences, workers=4)

In [ ]:
import tempfile

with tempfile.NamedTemporaryFile(prefix='gensim-model-', delete=False) as tmp:
    temporary_filepath = tmp.name
    model.save(temporary_filepath)
    #
    # The model is now safely stored in the filepath.
    # You can copy it to other machines, share it with others, etc.
    #
    # To load a saved model:
    #
    new_model = gensim.models.Word2Vec.load(temporary_filepath)